<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 80:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.1171875
Current benign train loss: 2.360161066055298

Current batch: 100
Current benign train accuracy: 0.265625
Current benign train loss: 1.8143000602722168

Current batch: 200
Current benign train accuracy: 0.3515625
Current benign train loss: 1.637120008468628

Current batch: 300
Current benign train accuracy: 0.4140625
Current benign train loss: 1.6378302574157715

Total benign train accuarcy: 34.746
Total benign train loss: 706.8949060440063

[ Test epoch: 0 ]

Test accuarcy: 43.74
Test average loss: 0.015490783882141113
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.453125
Current benign train loss: 1.5253549814224243

Current batch: 100
Current benign train accuracy: 0.4609375
Current benign train loss: 1.3404873609542847

Current batch: 200
Current benign train accuracy: 0.5078125
Current benign train loss: 1.4096347093582153

Current batch: 300
Current benign train accu


Current batch: 200
Current benign train accuracy: 0.9296875
Current benign train loss: 0.2681383490562439

Current batch: 300
Current benign train accuracy: 0.9296875
Current benign train loss: 0.21437038481235504

Total benign train accuarcy: 88.416
Total benign train loss: 132.54755015671253

[ Test epoch: 13 ]

Test accuarcy: 82.18
Test average loss: 0.005694511142373085
Model Saved!

[ Train epoch: 14 ]

Current batch: 0
Current benign train accuracy: 0.8828125
Current benign train loss: 0.3641297519207001

Current batch: 100
Current benign train accuracy: 0.8828125
Current benign train loss: 0.24207113683223724

Current batch: 200
Current benign train accuracy: 0.84375
Current benign train loss: 0.42644721269607544

Current batch: 300
Current benign train accuracy: 0.890625
Current benign train loss: 0.29773664474487305

Total benign train accuarcy: 88.956
Total benign train loss: 125.13086771965027

[ Test epoch: 14 ]

Test accuarcy: 84.43
Test average loss: 0.004840326926112175


Total benign train accuarcy: 91.868
Total benign train loss: 92.8359576612711

[ Test epoch: 26 ]

Test accuarcy: 87.97
Test average loss: 0.0036919092774391174
Model Saved!

[ Train epoch: 27 ]

Current batch: 0
Current benign train accuracy: 0.9609375
Current benign train loss: 0.14628586173057556

Current batch: 100
Current benign train accuracy: 0.9296875
Current benign train loss: 0.18440131843090057

Current batch: 200
Current benign train accuracy: 0.890625
Current benign train loss: 0.31926441192626953

Current batch: 300
Current benign train accuracy: 0.90625
Current benign train loss: 0.26928913593292236

Total benign train accuarcy: 92.326
Total benign train loss: 86.77591751515865

[ Test epoch: 27 ]

Test accuarcy: 86.32
Test average loss: 0.004357902440428734
Model Saved!

[ Train epoch: 28 ]

Current batch: 0
Current benign train accuracy: 0.9296875
Current benign train loss: 0.21466805040836334

Current batch: 100
Current benign train accuracy: 0.96875
Current benign t


Current batch: 0
Current benign train accuracy: 0.9375
Current benign train loss: 0.15854187309741974

Current batch: 100
Current benign train accuracy: 0.8984375
Current benign train loss: 0.30376383662223816

Current batch: 200
Current benign train accuracy: 0.9296875
Current benign train loss: 0.1352679580450058

Current batch: 300
Current benign train accuracy: 0.9296875
Current benign train loss: 0.1439724564552307

Total benign train accuarcy: 92.98
Total benign train loss: 78.53753091394901

[ Test epoch: 40 ]

Test accuarcy: 82.8
Test average loss: 0.005826605975627899
Model Saved!

[ Train epoch: 41 ]

Current batch: 0
Current benign train accuracy: 0.984375
Current benign train loss: 0.0736575573682785

Current batch: 100
Current benign train accuracy: 0.9375
Current benign train loss: 0.1351311355829239

Current batch: 200
Current benign train accuracy: 0.953125
Current benign train loss: 0.15547747910022736

Current batch: 300
Current benign train accuracy: 0.9609375
Curre


Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.014866269193589687

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.019650209695100784

Total benign train accuarcy: 98.766
Total benign train loss: 14.738295828457922

[ Test epoch: 53 ]

Test accuarcy: 94.14
Test average loss: 0.0020475675221532583
Model Saved!

[ Train epoch: 54 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.01226492878049612

Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.03255719318985939

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.007510298863053322

Current batch: 300
Current benign train accuracy: 0.9765625
Current benign train loss: 0.06403304636478424

Total benign train accuarcy: 99.01
Total benign train loss: 12.306921856012195

[ Test epoch: 54 ]

Test accuarcy: 94.11
Test average loss: 0.0020929946754127743
Model Saved!




Total benign train accuarcy: 99.714
Total benign train loss: 3.8192487959749997

[ Test epoch: 66 ]

Test accuarcy: 94.37
Test average loss: 0.002315033598802984
Model Saved!

[ Train epoch: 67 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0023678841535001993

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.006657508201897144

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.004925221670418978

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.013281366787850857

Total benign train accuarcy: 99.798
Total benign train loss: 3.374911946069915

[ Test epoch: 67 ]

Test accuarcy: 94.03
Test average loss: 0.0023527239050716163
Model Saved!

[ Train epoch: 68 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train loss: 0.031299907714128494

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0020


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.007273495197296143

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.004609796218574047

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.001117914100177586

Total benign train accuarcy: 99.888
Total benign train loss: 1.7779386800539214

[ Test epoch: 80 ]

Test accuarcy: 94.36
Test average loss: 0.0024029459109529854
Model Saved!

[ Train epoch: 81 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0014162373263388872

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.004589277785271406

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0011046761646866798

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.001488379784859717

Total benign train accuarcy: 99.912
Total benign train loss: 1.6126710234675556



Total benign train accuarcy: 99.97
Total benign train loss: 0.9686961027327925

[ Test epoch: 93 ]

Test accuarcy: 94.4
Test average loss: 0.002343545649200678
Model Saved!

[ Train epoch: 94 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0006948139052838087

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0012784721329808235

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.002422026824206114

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0019430358661338687

Total benign train accuarcy: 99.962
Total benign train loss: 1.0842041386349592

[ Test epoch: 94 ]

Test accuarcy: 94.35
Test average loss: 0.0023644451048225163
Model Saved!

[ Train epoch: 95 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.001045778626576066

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.001134354